## Week 6 : Large Language Models
```
- Generative Artificial Intelligence (Fall semester 2023)
- Professor: Muhammad Fahim
- Teaching Assistant: Ahmad Taha
```
<hr>

## Contents
```
1. Transformers (Implementing a transformer)
2. Self-Attention
3. Multi-headed attention
4. Positional Encoding

```

<hr>


# Transformers

* [Attention Is All You Need](https://arxiv.org/pdf/1706.03762.pdf) -- Original paper on attention

![](http://jalammar.github.io/images/t/The_transformer_encoder_decoder_stack.png)


In [ ]:
import torch
from torch import nn
import torch.optim as optim
import pandas as pd
import numpy as np

from torch.utils.data import DataLoader, TensorDataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### Transformer Encoder with Pytorch

In [ ]:
encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=32)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=3)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [ ]:
encoder_layer

TransformerEncoderLayer(
  (self_attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
  )
  (linear1): Linear(in_features=512, out_features=2048, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (linear2): Linear(in_features=2048, out_features=512, bias=True)
  (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (dropout1): Dropout(p=0.1, inplace=False)
  (dropout2): Dropout(p=0.1, inplace=False)
)

## Encoder

The encoder contains a self-attention layer – a layer that helps the encoder look at other words in the input sentence as it encodes a specific word. <br>
**The main goal is to efficiently encode the data**

![](http://jalammar.github.io/images/t/encoder_with_tensors.png)

## Self-Attention

**Keep in mind : The main goal is to encode the data in a much more efficient way** In other words is to create meaningful embeddings<br>
- As the model processes each word (each position in the input sequence), self attention allows it to look at other positions in the input sequence for clues that can help lead to a better encoding for this word.


**How does Self-Attention work?**

Steps:
1. For each word, we create a **`Query`** vector, a **`Key`** vector, and a **`Value`** vector.
  - What are the **`Query`** vector, a **`Key`** vector, and a **`Value`** vector? : They’re abstractions that are useful for calculating attention... They are a breakdown of the word embeddings
2. Calculating self-attention score from **`Query`** **`Key`** vector.
3. Divide the scores by 8 (This leads to having more stable gradients)
4. Pass the result through a softmax operation (softmax score determines how much each word will be expressed at this position)
5. Multiply each value vector by the softmax score
6. Sum up the weighted value vectors

### Step 1

For each word, we create a **`Query`** vector, a **`Key`** vector, and a **`Value`** vector.

![](http://jalammar.github.io/images/t/transformer_self_attention_vectors.png)

In [ ]:
# simple sequence = I am here today
simple_sequence_embedding = torch.rand((4, 512))

# Create weight matrices
Wq = torch.normal(0,0.5, (512, 7))
Wk = torch.normal(0,0.1, (512, 7))
Wv = torch.normal(0,0.2, (512, 7))

# Create key, query and value for each word in the senetence
queries = simple_sequence_embedding.mm(Wq) # self.q(embedding[0])
keys = simple_sequence_embedding.mm(Wk)
values = simple_sequence_embedding.mm(Wv)

In [ ]:
values

tensor([[-0.9431,  2.2429, -1.4976,  1.4453,  2.3804, -0.6668, -0.3153],
        [ 1.0690,  1.9116, -3.2465,  0.6282,  0.9388, -2.8510, -1.0129],
        [ 0.0309,  3.0505, -2.3856, -0.0125, -0.0168, -4.0641,  0.8794],
        [-1.6612,  2.0367, -0.5448,  1.7356,  1.2407, -2.7459, -1.3949]])

In [ ]:
simple_sequence_embedding

tensor([[0.3820, 0.4833, 0.7283,  ..., 0.7201, 0.4370, 0.6838],
        [0.0969, 0.7248, 0.2339,  ..., 0.3183, 0.0909, 0.3160],
        [0.3724, 0.0760, 0.2922,  ..., 0.5227, 0.4353, 0.1027],
        [0.6361, 0.9490, 0.9695,  ..., 0.5571, 0.9913, 0.0671]])

## Step 2

Calculating self-attention score from **`Query`** and **`Key`** vector

In [ ]:
scores = torch.mm(queries, keys.T)
scores

tensor([[47.3834, 40.2896, 38.8234, 79.1502],
        [48.7963, 41.6318, 35.0864, 78.1837],
        [45.2341, 41.0687, 32.6231, 71.3646],
        [44.4938, 33.5259, 17.7919, 77.1678]])

## Step 3
Divide the scores by 8 (This leads to having more stable gradients)

In [ ]:
scores = scores / 8
scores

tensor([[5.9229, 5.0362, 4.8529, 9.8938],
        [6.0995, 5.2040, 4.3858, 9.7730],
        [5.6543, 5.1336, 4.0779, 8.9206],
        [5.5617, 4.1907, 2.2240, 9.6460]])

## Step 4

Pass the result through a softmax operation

In [ ]:
scores = torch.nn.functional.softmax(scores, dim=1)
scores

tensor([[1.8253e-02, 7.5205e-03, 6.2611e-03, 9.6797e-01],
        [2.4405e-02, 9.9666e-03, 4.3976e-03, 9.6123e-01],
        [3.5695e-02, 2.1207e-02, 7.3789e-03, 9.3572e-01],
        [1.6478e-02, 4.1831e-03, 5.8525e-04, 9.7875e-01]])

## Step 5 & 6

* Multiply each value vector by the softmax score
* Sum up the weighted value vectors



In [ ]:
scores.shape, values.shape

(torch.Size([4, 4]), torch.Size([4, 7]))

In [ ]:
z = scores @ values
z

tensor([[-1.6170,  2.0458, -0.5940,  1.7110,  1.2514, -2.7170, -1.3581],
        [-1.6091,  2.0449, -0.6031,  1.7098,  1.2600, -2.7020, -1.3548],
        [-1.5652,  2.0489, -0.6497,  1.6889,  1.2657, -2.6836, -1.3315],
        [-1.6370,  2.0401, -0.5729,  1.7252,  1.2575, -2.7128, -1.3742]])

# Multi-headed attention

**GOAL**:
1. Expand the model’s ability to focus on different positions
2. Provide the attention layer multiple “representation subspaces”

**Attention with $N$ just means repeating self attention algorithm $N$ times and joining the results**


![](https://data-science-blog.com/wp-content/uploads/2022/01/mha_img_original.png)

**Multi-headed attention steps:**
1. Same as self-attention calculation, just n different times with different weight matrices
2. Condense the $N$ z metrices down into a single matrix by concatinating the matrices then multiply them by an additional weights matrix `WO`

Now the output z metrix is fed to the FFNN

In [ ]:
from torch import Tensor
import torch.nn.functional as f


def scaled_dot_product_attention(query, key, value):
  temp = query.bmm(key.transpose(1, 2))
  scale = query.size(-1) ** 0.5
  softmax = f.softmax(temp / scale, dim=-1)
  return softmax.bmm(value)

## Now lets make attention head

In [ ]:
import torch.nn as nn

class AttentionHead(nn.Module):
    def __init__(self, dim_in, dim_q, dim_k):
        super().__init__()
        # Fill in the missing parts of the constructor to initialize the query, key, and value linear transformations
        # dim_in is the input dimension, dim_q and dim_k are the output dimensions for the queries and keys/values respectively
        # Example: dim_in 512, dim_q = dim_k 64 in the paper

    def forward(self, query, key, value):
        # Implement the forward pass by calling scaled_dot_product_attention function
        # You need to transform the query, key, and value using the linear transformations defined in __init__
        # Fill in with the correct method calls

## Multi Head Attention

In [ ]:
class MultiHeadToAttention(nn.Module):
    def __init__(self, number_of_heads, dim_in, dim_q, dim_k):
        super().__init__()
        # Initialize heads as multi-AttentionHead instances
        # Initialize linear to combine the outputs of all heads into a single output

    def forward(self, query: Tensor, key: Tensor, value: Tensor):
        # Concatenate outputs from all heads and apply the final linear transformation
        return z

## Positional Encoding

A way to account for the order of the words in the input sequence. A transformer adds a vector to each input embedding which helps it determine the position of each word. <br>
**Goal** : preserving information about the order of tokens  

Positional Encoding they can either be learned or fixed a priori.

Proposed approach from original paper : describe a simple scheme for fixed positional encodings based on sine and cosine functions

![](https://miro.medium.com/v2/resize:fit:640/format:webp/1*C3a9RL6-SFC6fW8NGpJg5A.png)

In [ ]:
def position_encoding(seq_len, dim_model, device):
    # Define the position tensor 'pos' with dimensions appropriate for sequence length
    pos =

    # Define the dimension tensor 'dim' suitable for the model dimensions
    dim =

    # Calculate the phase using the position and dimension tensors
    phase =

    # Return the sinusoidal position encoding
    return   # Complete this line to select sin or cos based on even/odd index

## Encoder Feed Forward

In [ ]:
def feed_forward(dim_input = 512, dim_feedforward = 2048):
  return nn.Sequential(nn.Linear(dim_input, dim_feedforward),
                       nn.ReLU(),
                       nn.Linear(dim_feedforward, dim_input)
                       )

## Encoder Residual

From the original paper the author implementation

In [ ]:
class Residual(nn.Module):
  def __init__(self, sublayer, dimension, dropout = 0.1):
    super().__init__()
    self.sublayer = sublayer
    self.norm = nn.LayerNorm(dimension)
    self.dropout = nn.Dropout(dropout)

  def forward(self, *tensors):
    return self.norm(tensors[0] + self.dropout(self.sublayer(*tensors)))

## Putting all together on encoder side

![](http://jalammar.github.io/images/t/transformer_resideual_layer_norm_2.png)

## Putting the Encoder layer together

In [ ]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, dim_model=512, num_heads=6, dim_feedforward=2048, dropout=0.1):
        super().__init__()
        # Define dimensions for query and key based on model dimension and number of heads

        # Initialize the MultiHeadAttention component with a residual connection and dropout
        self.attention =

        # Initialize the feedforward component with a residual connection and dropout
        self.feed_forward =

    def forward(self, src):
        # Apply the attention mechanism to the source data
        src =

        # Apply the feedforward network to the output of the attention mechanism
        return

## Putting together transfomer Encoder part

In [ ]:
class TransformerEncoder(nn.Module):
    def __init__(self, num_layers=12, dim_model=512, num_heads=4, dim_feedforward=2048, dropout=0.1):
        super().__init__()
        # Initialize a list of TransformerEncoderLayer instances
        self.layers =

    def forward(self, src):
        # Retrieve the sequence length and dimension from the source tensor
        seq_len, dimension =

        # Add position encoding to the source tensor
        src +=

        # Process each layer in the transformer encoder

        return src

# The Decoder Side

The encoder start by processing the input sequence. The output of the top encoder is then transformed into a set of attention vectors K and V. These are to be used by each decoder.


![](https://jalammar.github.io/images/t/transformer_resideual_layer_norm_3.png)

## Decoder layer

The “Encoder-Decoder Attention” layer works just like multiheaded self-attention, except it creates its Queries matrix from the layer below it, and takes the Keys and Values matrix from the output of the encoder stack.

**Task**: implement the decoder layer

In [ ]:
class TransformerDecoderLayer(nn.Module):
    def __init__(
        self,
        dim_model: int = 512,
        num_heads: int = 6,
        dim_feedforward: int = 2048,
        dropout: float = 0.1,
    ):
        super().__init__()
        # init dim_q and dim_k as in the encoder
        # Initialize the first self-attention layer with a residual connection
        self.attention_1 =

        # Initialize the second attention layer for interaction with the encoder output
        self.attention_2 =

        # Initialize the feed-forward network
        self.feed_forward =

    def forward(self, tgt: Tensor, memory: Tensor) -> Tensor:
        # Self-attention mechanism
        tgt =

        # Cross-attention mechanism where the decoder attends to the encoder's output
        tgt =

        # Pass through the feed-forward network
        return

## Full Transfomer Decoder

**Task**: implement the transfomer decoder part class

In [ ]:
class TransformerDecoder(nn.Module):
    def __init__(
        self,
        num_layers: int = 6,
        dim_model: int = 512,
        num_heads: int = 8,
        dim_feedforward: int = 2048,
        dropout: float = 0.1,
    ):
        super().__init__()
        # Initialize laters from TransformerDecoderLayer instances
        self.layers =
        # Define the output linear transformation
        self.linear =

    def forward(self, tgt: Tensor, memory: Tensor) -> Tensor:
        # Calculate sequence length and dimension from the target tensor
        seq_len, dimension =

        # Add position encoding to the target tensor
        tgt +=

        # Process each layer in the transformer decoder
        # Code here

        # Apply a softmax to the output of the final linear layer
        return

## Full Transfomer model

**Task**:
1. Assembly a full transfomer (Encoder + Decoder)
2. Implement the Transfomer training loop
3. Using dataset of your choice train the transformer just for one epoch


In [ ]:
class Transfomer(nn.Module):
  def __init__(self, ... ):
    super().__init__()
    pass

  def forward(self):
    pass